In [1]:
# notebook source: https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.pyd

In [2]:
# network parameters
kernel_size = (5,5)
pool_size=(2,2)
filters_layer1 = 32
filters_layer2 = 64
#activation='relu'
dropout = 0.5
units_dense = 1024

# train parameters
batch_size = 128
epochs = 10
#loss=keras.losses.categorical_crossentropy
#optimizer=adam()
#metrics=['accuracy']


In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5206470572164568557
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1547193548
locality {
  bus_id: 1
}
incarnation: 3496290125719408921
physical_device_desc: "device: 0, name: GeForce 940MX, pci bus id: 0000:01:00.0, compute capability: 5.0"
]


In [4]:
'''Trains a simple convnet on the MNIST dataset.

Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras import backend as K
from keras.utils import np_utils

# input image dimensions
img_rows, img_cols = 28, 28

num_classes = 10

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_dim_ordering() == 'th':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(filters_layer1, kernel_size=kernel_size[0],
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Conv2D(filters_layer2, kernel_size, activation='relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Flatten())
model.add(Dense(units_dense, activation='relu'))
model.add(Dropout(dropout))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Using TensorFlow backend.


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 33s 548us/step - loss: 0.1727 - acc: 0.9464 - val_loss: 0.0399 - val_acc: 0.9872
Epoch 2/10
60000/60000 [==============================] - 29s 487us/step - loss: 0.0464 - acc: 0.9856 - val_loss: 0.0275 - val_acc: 0.9910
Epoch 3/10
60000/60000 [==============================] - 29s 480us/step - loss: 0.0335 - acc: 0.9896 - val_loss: 0.0247 - val_acc: 0.9910
Epoch 4/10
60000/60000 [==============================] - 28s 474us/step - loss: 0.0257 - acc: 0.9914 - val_loss: 0.0228 - val_acc: 0.9926
Epoch 5/10
60000/60000 [==============================] - 28s 474us/step - loss: 0.0214 - acc: 0.9933 - val_loss: 0.0237 - val_acc: 0.9928
Epoch 6/10
60000/60000 [==============================] - 29s 476us/step - loss: 0.0165 - acc: 0.9947 - val_loss: 0.0257 - val_acc: 0.9921
Epoch 7/10
60000/60000 [===================

In [5]:
# save and load model
# source: https://machinelearningmastery.com/save-load-keras-deep-learning-models/

In [6]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [7]:
# later...
from keras.models import model_from_json
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
score = loaded_model.evaluate(x_test, y_test, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

Loaded model from disk
acc: 99.27%


In [8]:
# deploy and measure predict speed

import time
start = time.time()
import numpy as np
model.predict(x_test[:10,:])

end = time.time()
print(10/(end-start))

171.9554932395313
